In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Keys
from config import (api_key)
c = Census(api_key)

In [2]:

# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
# Note the addition of "B23025_005E" for unemployment count
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E", 
                          "C24010_007E",
                         'C24010_043E',
                         "B02001_002E", 
                          "B02001_003E", 
                          "B02001_004E", 
                          "B02001_005E",  
                          "B02001_008E", 
                          "B03001_003E"), {'for': 'zip code tabulation area:*'},)

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "C24010_007E": "Men employed in STEM fields",
                                      'C24010_043E':"Women employed in STEM fields",
                                      "NAME": "Name", 
                                      "B02001_002E": 'White',
                                      "B02001_003E":'African American',
                                      "B02001_004E":'Indian', 
                                      "B02001_005E":'Asian',
                                      "B03001_003E":'Hispanic',
                                      "B02001_008E":'Two or more Races',
                                      "zip code tabulation area": "ZIP Code"})


In [3]:
census_pd["Indian"]

0          25.0
1           0.0
2         115.0
3          18.0
4           0.0
          ...  
33115       0.0
33116       0.0
33117       0.0
33118       1.0
33119    1342.0
Name: Indian, Length: 33120, dtype: float64

In [4]:
#Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * census_pd["Poverty Count"] / census_pd["Population"]

# Add in Employment Rate (Employment Count / Population)

# census_pd["Unemployment Rate"] = 100 * census_pd["Unemployment Count"] / census_pd["Population"]

# Population in STEM
census_pd["Men in STEM_Rate"] = 100 * census_pd["Men employed in STEM fields"] / census_pd["Population"]


census_pd["Women in STEM_Rate"] = 100 * census_pd["Women employed in STEM fields"] / census_pd["Population"]


#Population and Races

census_pd["White_%"] = 100 * census_pd["White"] / census_pd["Population"]

census_pd["African American_%"] = 100 * census_pd["African American"] / census_pd["Population"]

census_pd["Indian_%"] = 100 * census_pd["Indian"] / census_pd["Population"]

census_pd["Asian_%"] = 100 * census_pd["Asian"] / census_pd["Population"]

census_pd["Hispanic_%"] = 100 * census_pd["Hispanic"] / census_pd["Population"]

census_pd["Two or More Races_%"] = 100 * census_pd["Two or more Races"] / census_pd["Population"]


# Final DataFrame
census_pd = census_pd[["ZIP Code", "Population", "Median Age","Household Income", "Unemployment Count",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", 
                       "Men in STEM_Rate","Women in STEM_Rate",
                       "Men employed in STEM fields",
                       "Women employed in STEM fields", "White_%", "African American_%","Indian_%",'Asian_%', 'Hispanic_%',
                      "Two or More Races_%"]]

census_pd

,ZIP Code,Population,Median Age,Household Income,Unemployment Count,Per Capita Income,Poverty Count,Poverty Rate,Men in STEM_Rate,Women in STEM_Rate,Men employed in STEM fields,Women employed in STEM fields,White_%,African American_%,Indian_%,Asian_%,Hispanic_%,Two or More Races_%
0,00601,17242.0,40.5,13092.0,2316.0,6999.0,10772.0,62.475351,0.162394,0.121796,28.0,21.0,75.548080,0.840970,0.144995,0.017399,99.663612,0.661176
1,00602,38442.0,42.3,16358.0,1927.0,9277.0,19611.0,51.014515,0.910463,0.210707,350.0,81.0,79.415743,2.783414,0.000000,0.000000,93.452474,13.794808
2,00603,48814.0,41.1,16603.0,3124.0,11307.0,24337.0,49.856599,0.952596,0.176179,465.0,86.0,76.473962,3.953784,0.235588,0.745688,97.412628,2.167411
3,00606,6437.0,43.3,12832.0,230.0,5943.0,4163.0,64.672984,0.000000,0.000000,0.0,0.0,40.810937,2.314743,0.279633,0.000000,99.844648,1.941898
4,00610,27073.0,42.1,19309.0,1290.0,10220.0,11724.0,43.305138,0.546670,0.454327,148.0,123.0,75.540206,2.570827,0.000000,0.000000,96.195472,11.280612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33115,87515,363.0,44.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.446281,0.000000,0.000000,0.000000,100.000000,0.000000
33116,87518,9.0,-666666666.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000
33117,87511,2896.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.436464,1.277624,0.000000,0.000000,87.430939,0.000000
33118,87578,245.0,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.122449,0.000000,0.408163,0.408163,82.448980,3.265306


In [5]:
census_pd["Unemployment Rate"] = 100 * census_pd["Unemployment Count"] / census_pd["Population"]
del census_pd["Unemployment Count"]

In [6]:
census_pd

,ZIP Code,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Men in STEM_Rate,Women in STEM_Rate,Men employed in STEM fields,Women employed in STEM fields,White_%,African American_%,Indian_%,Asian_%,Hispanic_%,Two or More Races_%,Unemployment Rate
0,00601,17242.0,40.5,13092.0,6999.0,10772.0,62.475351,0.162394,0.121796,28.0,21.0,75.548080,0.840970,0.144995,0.017399,99.663612,0.661176,13.432316
1,00602,38442.0,42.3,16358.0,9277.0,19611.0,51.014515,0.910463,0.210707,350.0,81.0,79.415743,2.783414,0.000000,0.000000,93.452474,13.794808,5.012746
2,00603,48814.0,41.1,16603.0,11307.0,24337.0,49.856599,0.952596,0.176179,465.0,86.0,76.473962,3.953784,0.235588,0.745688,97.412628,2.167411,6.399803
3,00606,6437.0,43.3,12832.0,5943.0,4163.0,64.672984,0.000000,0.000000,0.0,0.0,40.810937,2.314743,0.279633,0.000000,99.844648,1.941898,3.573093
4,00610,27073.0,42.1,19309.0,10220.0,11724.0,43.305138,0.546670,0.454327,148.0,123.0,75.540206,2.570827,0.000000,0.000000,96.195472,11.280612,4.764895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33115,87515,363.0,44.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.446281,0.000000,0.000000,0.000000,100.000000,0.000000,NaN
33116,87518,9.0,-666666666.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,NaN
33117,87511,2896.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.436464,1.277624,0.000000,0.000000,87.430939,0.000000,NaN
33118,87578,245.0,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.122449,0.000000,0.408163,0.408163,82.448980,3.265306,NaN


In [7]:
census_pd["ZIP Code"]=census_pd["ZIP Code"].astype(int)

In [8]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
census_pd.to_csv("../Resources/census_data_for_medicare.csv", encoding="utf-8", index=False)

In [9]:
# Read in the csv containing coordinates
Lat_long_df = pd.read_csv("../Resources/us-zip-code-latitude-and-longitude (1).csv", sep = ';', engine = 'python')
Lat_long_df

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,"geopoint,"
0,48834,Fenwick,MI,43.141649,-85.04948,-5,1,"43.141649,-85.04948"
1,55304,Andover,MN,45.254715,-93.28652,-6,1,"45.254715,-93.28652"
2,55422,Minneapolis,MN,45.014764,-93.33965,-6,1,"45.014764,-93.33965"
3,29079,Lydia,SC,34.296064,-80.11319,-5,1,"34.296064,-80.11319"
4,29390,Duncan,SC,34.888237,-81.96902,-5,1,"34.888237,-81.96902"
...,...,...,...,...,...,...,...,...
43186,50157,Malcom,IA,41.738800,-92.56155,-6,1,"41.7388,-92.56155"
43187,65212,Columbia,MO,38.937608,-92.33043,-6,1,"38.937608,-92.33043"
43188,11935,Cutchogue,NY,41.012868,-72.47230,-5,1,"41.012868,-72.4723"
43189,63539,Ethel,MO,39.893011,-92.74503,-6,1,"39.893011,-92.74503"


In [10]:
Lat_long_df=Lat_long_df.rename(columns={'Zip':'ZIP Code'})

In [11]:
Lat_long_df = Lat_long_df[['ZIP Code', 'City', 'State', 'Latitude', "Longitude"]]
Lat_long_df


,ZIP Code,City,State,Latitude,Longitude
0,48834,Fenwick,MI,43.141649,-85.04948
1,55304,Andover,MN,45.254715,-93.28652
2,55422,Minneapolis,MN,45.014764,-93.33965
3,29079,Lydia,SC,34.296064,-80.11319
4,29390,Duncan,SC,34.888237,-81.96902
...,...,...,...,...,...
43186,50157,Malcom,IA,41.738800,-92.56155
43187,65212,Columbia,MO,38.937608,-92.33043
43188,11935,Cutchogue,NY,41.012868,-72.47230
43189,63539,Ethel,MO,39.893011,-92.74503


In [12]:
merged_df= pd.merge(census_pd, Lat_long_df, on='ZIP Code')
merged_df

,ZIP Code,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Men in STEM_Rate,Women in STEM_Rate,Men employed in STEM fields,...,African American_%,Indian_%,Asian_%,Hispanic_%,Two or More Races_%,Unemployment Rate,City,State,Latitude,Longitude
0,601,17242.0,40.5,13092.0,6999.0,10772.0,62.475351,0.162394,0.121796,28.0,...,0.840970,0.144995,0.017399,99.663612,0.661176,13.432316,Adjuntas,PR,18.180103,-66.74947
1,602,38442.0,42.3,16358.0,9277.0,19611.0,51.014515,0.910463,0.210707,350.0,...,2.783414,0.000000,0.000000,93.452474,13.794808,5.012746,Aguada,PR,18.363285,-67.18024
2,603,48814.0,41.1,16603.0,11307.0,24337.0,49.856599,0.952596,0.176179,465.0,...,3.953784,0.235588,0.745688,97.412628,2.167411,6.399803,Aguadilla,PR,18.448619,-67.13422
3,606,6437.0,43.3,12832.0,5943.0,4163.0,64.672984,0.000000,0.000000,0.0,...,2.314743,0.279633,0.000000,99.844648,1.941898,3.573093,Maricao,PR,18.182151,-66.95880
4,610,27073.0,42.1,19309.0,10220.0,11724.0,43.305138,0.546670,0.454327,148.0,...,2.570827,0.000000,0.000000,96.195472,11.280612,4.764895,Anasco,PR,18.288319,-67.13604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32898,87515,363.0,44.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,100.000000,0.000000,NaN,Canjilon,NM,36.476265,-106.40996
32899,87518,9.0,-666666666.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,100.000000,0.000000,NaN,Cebolla,NM,36.522263,-106.54918
32900,87511,2896.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.277624,0.000000,0.000000,87.430939,0.000000,NaN,Alcalde,NM,36.097705,-106.04254
32901,87578,245.0,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.408163,0.408163,82.448980,3.265306,NaN,Truchas,NM,36.036721,-105.78702


In [13]:
merged_df.count()

ZIP Code                         32903
Population                       32903
Median Age                       32903
Household Income                 32868
Per Capita Income                32560
Poverty Count                    32868
Poverty Rate                     32550
Men in STEM_Rate                 32550
Women in STEM_Rate               32550
Men employed in STEM fields      32868
Women employed in STEM fields    32868
White_%                          32584
African American_%               32584
Indian_%                         32584
Asian_%                          32584
Hispanic_%                       32584
Two or More Races_%              32584
Unemployment Rate                32550
City                             32903
State                            32903
Latitude                         32903
Longitude                        32903
dtype: int64

In [14]:
merged_df=merged_df.dropna()
merged_df

,ZIP Code,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Men in STEM_Rate,Women in STEM_Rate,Men employed in STEM fields,...,African American_%,Indian_%,Asian_%,Hispanic_%,Two or More Races_%,Unemployment Rate,City,State,Latitude,Longitude
0,601,17242.0,40.5,13092.0,6999.0,10772.0,62.475351,0.162394,0.121796,28.0,...,0.840970,0.144995,0.017399,99.663612,0.661176,13.432316,Adjuntas,PR,18.180103,-66.74947
1,602,38442.0,42.3,16358.0,9277.0,19611.0,51.014515,0.910463,0.210707,350.0,...,2.783414,0.000000,0.000000,93.452474,13.794808,5.012746,Aguada,PR,18.363285,-67.18024
2,603,48814.0,41.1,16603.0,11307.0,24337.0,49.856599,0.952596,0.176179,465.0,...,3.953784,0.235588,0.745688,97.412628,2.167411,6.399803,Aguadilla,PR,18.448619,-67.13422
3,606,6437.0,43.3,12832.0,5943.0,4163.0,64.672984,0.000000,0.000000,0.0,...,2.314743,0.279633,0.000000,99.844648,1.941898,3.573093,Maricao,PR,18.182151,-66.95880
4,610,27073.0,42.1,19309.0,10220.0,11724.0,43.305138,0.546670,0.454327,148.0,...,2.570827,0.000000,0.000000,96.195472,11.280612,4.764895,Anasco,PR,18.288319,-67.13604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32862,99922,330.0,39.5,34028.0,18213.0,129.0,39.090909,3.636364,0.909091,12.0,...,1.212121,71.818182,0.000000,0.000000,5.757576,3.333333,Hydaburg,AK,55.209339,-132.82545
32864,99925,927.0,43.6,57375.0,25840.0,172.0,18.554477,0.000000,0.755124,0.0,...,0.323625,39.482201,0.000000,2.804746,11.326861,9.600863,Klawock,AK,55.555164,-133.07316
32865,99926,1635.0,34.5,53409.0,22453.0,235.0,14.373089,0.550459,0.000000,9.0,...,0.000000,74.006116,1.345566,7.094801,8.990826,8.440367,Metlakatla,AK,55.123897,-131.56883
32866,99927,38.0,55.5,-666666666.0,13658.0,28.0,73.684211,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Point Baker,AK,56.337957,-133.60689


In [30]:
del merged_df['City']

In [31]:
del merged_df['State']

In [32]:
merged_df

,ZIP Code,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Men in STEM_Rate,Women in STEM_Rate,Men employed in STEM fields,Women employed in STEM fields,White_%,African American_%,Indian_%,Asian_%,Hispanic_%,Two or More Races_%,Unemployment Rate,Latitude,Longitude
0,601,17242.0,40.5,13092.0,6999.0,10772.0,62.475351,0.162394,0.121796,28.0,21.0,75.548080,0.840970,0.144995,0.017399,99.663612,0.661176,13.432316,18.180103,-66.74947
1,602,38442.0,42.3,16358.0,9277.0,19611.0,51.014515,0.910463,0.210707,350.0,81.0,79.415743,2.783414,0.000000,0.000000,93.452474,13.794808,5.012746,18.363285,-67.18024
2,603,48814.0,41.1,16603.0,11307.0,24337.0,49.856599,0.952596,0.176179,465.0,86.0,76.473962,3.953784,0.235588,0.745688,97.412628,2.167411,6.399803,18.448619,-67.13422
3,606,6437.0,43.3,12832.0,5943.0,4163.0,64.672984,0.000000,0.000000,0.0,0.0,40.810937,2.314743,0.279633,0.000000,99.844648,1.941898,3.573093,18.182151,-66.95880
4,610,27073.0,42.1,19309.0,10220.0,11724.0,43.305138,0.546670,0.454327,148.0,123.0,75.540206,2.570827,0.000000,0.000000,96.195472,11.280612,4.764895,18.288319,-67.13604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32862,99922,330.0,39.5,34028.0,18213.0,129.0,39.090909,3.636364,0.909091,12.0,3.0,12.424242,1.212121,71.818182,0.000000,0.000000,5.757576,3.333333,55.209339,-132.82545
32864,99925,927.0,43.6,57375.0,25840.0,172.0,18.554477,0.000000,0.755124,0.0,7.0,48.112190,0.323625,39.482201,0.000000,2.804746,11.326861,9.600863,55.555164,-133.07316
32865,99926,1635.0,34.5,53409.0,22453.0,235.0,14.373089,0.550459,0.000000,9.0,0.0,14.617737,0.000000,74.006116,1.345566,7.094801,8.990826,8.440367,55.123897,-131.56883
32866,99927,38.0,55.5,-666666666.0,13658.0,28.0,73.684211,0.000000,0.000000,0.0,0.0,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,56.337957,-133.60689


In [33]:
merged_df.to_csv("../Resources/to_be_used.csv", encoding="utf-8", index=False)

In [34]:
# Read in the csv containing coordinates
payment_df = pd.read_csv("../Resources/Payment_and_Value_of_Care-Hospital (2).csv")
payment_df

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Payment Measure ID,Payment Measure Name,...,Payment,Lower Estimate,Higher Estimate,Payment Footnote,Value of Care Display ID,Value of Care Display Name,Value of Care Category,Value of Care Footnote,Start Date,End Date
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_AMI,Payment for heart attack patients,...,"$24,934","$23,490","$26,429",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,7/1/16,6/30/19
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_HF,Payment for heart failure patients,...,"$17,854","$17,061","$18,676",NaN,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,NaN,7/1/16,6/30/19
2,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_30_PN,Payment for pneumonia patients,...,"$20,216","$19,186","$21,271",NaN,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,NaN,7/1/16,6/30/19
3,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,"$22,216","$21,191","$23,313",NaN,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment,NaN,4/1/16,3/31/19
4,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,PAYM_30_AMI,Payment for heart attack patients,...,"$24,742","$22,016","$27,967",NaN,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,NaN,7/1/16,6/30/19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18835,670136,BAYLOR SCOTT & WHITE MEDICAL CENTER- AUSTIN,5245 W US 290,AUSTIN,TX,78735,TRAVIS,(512) 654-2100,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,...,Not Available,Not Available,Not Available,5.0,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Not Available,5.0,4/1/16,3/31/19
18836,670143,ASCENSION SETON BASTROP,"630 HIGWAY 71 W,",BASTROP,TX,78602,BASTROP,(737) 881-7400,PAYM_30_AMI,Payment for heart attack patients,...,Not Available,Not Available,Not Available,19.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Not Available,19.0,7/1/16,6/30/19
18837,670143,ASCENSION SETON BASTROP,"630 HIGWAY 71 W,",BASTROP,TX,78602,BASTROP,(737) 881-7400,PAYM_30_HF,Payment for heart failure patients,...,Not Available,Not Available,Not Available,19.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Not Available,19.0,7/1/16,6/30/19
18838,670143,ASCENSION SETON BASTROP,"630 HIGWAY 71 W,",BASTROP,TX,78602,BASTROP,(737) 881-7400,PAYM_30_PN,Payment for pneumonia patients,...,Not Available,Not Available,Not Available,19.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Not Available,19.0,7/1/16,6/30/19


In [35]:
payment_df.columns

Index(['Facility ID', 'Facility Name', 'Address', 'City', 'State', 'ZIP Code',
       'County Name', 'Phone Number', 'Payment Measure ID',
       'Payment Measure Name', 'Payment Category', 'Denominator', 'Payment',
       'Lower Estimate', 'Higher Estimate', 'Payment Footnote',
       'Value of Care Display ID', 'Value of Care Display Name',
       'Value of Care Category', 'Value of Care Footnote', 'Start Date',
       'End Date'],
      dtype='object')

In [36]:
# merged_df[merged_df['Value of Care Category'] != 'Not Available']

cleaned_df=payment_df[payment_df['Value of Care Category'] != "Not Available"]
cleaned_df= cleaned_df[['Facility Name', 'City', 'State', 'ZIP Code', 'Payment Measure ID', 'Payment Measure Name', 'Payment Category', 'Payment',
       'Value of Care Display ID', 'Value of Care Display Name',
       'Value of Care Category']]
cleaned_df

,Facility Name,City,State,ZIP Code,Payment Measure ID,Payment Measure Name,Payment Category,Payment,Value of Care Display ID,Value of Care Display Name,Value of Care Category
0,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,PAYM_30_AMI,Payment for heart attack patients,No Different Than the National Average Payment,"$24,934",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
1,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,PAYM_30_HF,Payment for heart failure patients,No Different Than the National Average Payment,"$17,854",MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment
2,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,PAYM_30_PN,Payment for pneumonia patients,Greater Than the National Average Payment,"$20,216",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment
3,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,Greater Than the National Average Payment,"$22,216",COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment
4,MARSHALL MEDICAL CENTERS,BOAZ,AL,35957,PAYM_30_AMI,Payment for heart attack patients,No Different Than the National Average Payment,"$24,742",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
...,...,...,...,...,...,...,...,...,...,...,...
18786,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,EL PASO,TX,79911,PAYM_30_PN,Payment for pneumonia patients,No Different Than the National Average Payment,"$20,221",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment
18792,HOUSTON METHODIST THE WOODLANDS HOSPITAL,THE WOODLANDS,TX,77385,PAYM_30_AMI,Payment for heart attack patients,No Different Than the National Average Payment,"$27,138",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
18793,HOUSTON METHODIST THE WOODLANDS HOSPITAL,THE WOODLANDS,TX,77385,PAYM_30_HF,Payment for heart failure patients,Greater Than the National Average Payment,"$21,647",MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Higher Payment
18794,HOUSTON METHODIST THE WOODLANDS HOSPITAL,THE WOODLANDS,TX,77385,PAYM_30_PN,Payment for pneumonia patients,No Different Than the National Average Payment,"$19,409",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Better Mortality and Average Payment


In [37]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12135 entries, 0 to 18795
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Facility Name               12135 non-null  object
 1   City                        12135 non-null  object
 2   State                       12135 non-null  object
 3   ZIP Code                    12135 non-null  int64 
 4   Payment Measure ID          12135 non-null  object
 5   Payment Measure Name        12135 non-null  object
 6   Payment Category            12135 non-null  object
 7   Payment                     12135 non-null  object
 8   Value of Care Display ID    12135 non-null  object
 9   Value of Care Display Name  12135 non-null  object
 10  Value of Care Category      12135 non-null  object
dtypes: int64(1), object(10)
memory usage: 1.1+ MB


In [38]:
cleaned_df['Value of Care Category'].unique()

array(['Average Mortality and Average Payment',
       'Average Mortality and Higher Payment',
       'Average Complications and Higher Payment',
       'Worse Mortality and Average Payment',
       'Worse Mortality and Lower Payment',
       'Average Complications and Average Payment',
       'Average Mortality and Lower Payment',
       'Average Complications and Lower Payment',
       'Better Mortality and Average Payment',
       'Worse Complications and Higher Payment',
       'Better Mortality and Lower Payment',
       'Worse Mortality and Higher Payment',
       'Worse Complications and Lower Payment',
       'Better Mortality and Higher Payment',
       'Better Complications and Lower Payment',
       'Better Complications and Average Payment',
       'Better Complications and Higher Payment',
       'Worse Complications and Average Payment'], dtype=object)

In [39]:
cleaned_df['Payment Category'].unique()

array(['No Different Than the National Average Payment',
       'Greater Than the National Average Payment',
       'Less Than the National Average Payment'], dtype=object)

In [40]:
cleaned_df['Payment Measure Name'].unique()

array(['Payment for heart attack patients',
       'Payment for heart failure patients',
       'Payment for pneumonia patients',
       'Payment for hip/knee replacement patients'], dtype=object)

In [41]:
city=pd.DataFrame(cleaned_df.groupby('State')['City'].agg('count')).reset_index()
city

,State,City
0,AK,36
1,AL,226
2,AR,189
3,AZ,203
4,CA,912
5,CO,200
6,CT,99
7,DC,21
8,DE,24
9,FL,625


In [42]:
cleaned_df['Payment']=cleaned_df['Payment'].str.replace('$', '', regex=True)
cleaned_df

,Facility Name,City,State,ZIP Code,Payment Measure ID,Payment Measure Name,Payment Category,Payment,Value of Care Display ID,Value of Care Display Name,Value of Care Category
0,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,PAYM_30_AMI,Payment for heart attack patients,No Different Than the National Average Payment,"24,934",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
1,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,PAYM_30_HF,Payment for heart failure patients,No Different Than the National Average Payment,"17,854",MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment
2,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,PAYM_30_PN,Payment for pneumonia patients,Greater Than the National Average Payment,"20,216",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment
3,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,Greater Than the National Average Payment,"22,216",COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment
4,MARSHALL MEDICAL CENTERS,BOAZ,AL,35957,PAYM_30_AMI,Payment for heart attack patients,No Different Than the National Average Payment,"24,742",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
...,...,...,...,...,...,...,...,...,...,...,...
18786,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,EL PASO,TX,79911,PAYM_30_PN,Payment for pneumonia patients,No Different Than the National Average Payment,"20,221",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment
18792,HOUSTON METHODIST THE WOODLANDS HOSPITAL,THE WOODLANDS,TX,77385,PAYM_30_AMI,Payment for heart attack patients,No Different Than the National Average Payment,"27,138",MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment
18793,HOUSTON METHODIST THE WOODLANDS HOSPITAL,THE WOODLANDS,TX,77385,PAYM_30_HF,Payment for heart failure patients,Greater Than the National Average Payment,"21,647",MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Higher Payment
18794,HOUSTON METHODIST THE WOODLANDS HOSPITAL,THE WOODLANDS,TX,77385,PAYM_30_PN,Payment for pneumonia patients,No Different Than the National Average Payment,"19,409",MORT_PAYM_30_PN,Value of Care Pneumonia measure,Better Mortality and Average Payment


In [43]:
cleaned_df['Payment']=cleaned_df['Payment'].str.replace(r'\D', '')

In [55]:
cleaned_df['Payment'] = pd.to_numeric(cleaned_df['Payment'], downcast='float')
cleaned_df

,Facility Name,City,State,ZIP Code,Payment Measure ID,Payment Measure Name,Payment Category,Payment,Value of Care Display ID,Value of Care Display Name,Value of Care Category,value_code
0,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,PAYM_30_AMI,Payment for heart attack patients,No Different Than the National Average Payment,24934.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,AMI
1,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,PAYM_30_HF,Payment for heart failure patients,No Different Than the National Average Payment,17854.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,HF
2,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,PAYM_30_PN,Payment for pneumonia patients,Greater Than the National Average Payment,20216.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,PN
3,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,Greater Than the National Average Payment,22216.0,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment,KNEE
4,MARSHALL MEDICAL CENTERS,BOAZ,AL,35957,PAYM_30_AMI,Payment for heart attack patients,No Different Than the National Average Payment,24742.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,AMI
...,...,...,...,...,...,...,...,...,...,...,...,...
18786,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,EL PASO,TX,79911,PAYM_30_PN,Payment for pneumonia patients,No Different Than the National Average Payment,20221.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment,PN
18792,HOUSTON METHODIST THE WOODLANDS HOSPITAL,THE WOODLANDS,TX,77385,PAYM_30_AMI,Payment for heart attack patients,No Different Than the National Average Payment,27138.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,AMI
18793,HOUSTON METHODIST THE WOODLANDS HOSPITAL,THE WOODLANDS,TX,77385,PAYM_30_HF,Payment for heart failure patients,Greater Than the National Average Payment,21647.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Higher Payment,HF
18794,HOUSTON METHODIST THE WOODLANDS HOSPITAL,THE WOODLANDS,TX,77385,PAYM_30_PN,Payment for pneumonia patients,No Different Than the National Average Payment,19409.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Better Mortality and Average Payment,PN


In [56]:
cleaned_df['value_code'] = cleaned_df['Value of Care Display ID'].apply(lambda x: x.split('_')[-1])
cleaned_df

,Facility Name,City,State,ZIP Code,Payment Measure ID,Payment Measure Name,Payment Category,Payment,Value of Care Display ID,Value of Care Display Name,Value of Care Category,value_code
0,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,PAYM_30_AMI,Payment for heart attack patients,No Different Than the National Average Payment,24934.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,AMI
1,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,PAYM_30_HF,Payment for heart failure patients,No Different Than the National Average Payment,17854.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Average Payment,HF
2,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,PAYM_30_PN,Payment for pneumonia patients,Greater Than the National Average Payment,20216.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Higher Payment,PN
3,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,PAYM_90_HIP_KNEE,Payment for hip/knee replacement patients,Greater Than the National Average Payment,22216.0,COMP_PAYM_90_HIP_KNEE,Value of Care hip/knee replacement,Average Complications and Higher Payment,KNEE
4,MARSHALL MEDICAL CENTERS,BOAZ,AL,35957,PAYM_30_AMI,Payment for heart attack patients,No Different Than the National Average Payment,24742.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,AMI
...,...,...,...,...,...,...,...,...,...,...,...,...
18786,THE HOSPITALS OF PROVIDENCE TRANSMOUNTAIN CAMPUS,EL PASO,TX,79911,PAYM_30_PN,Payment for pneumonia patients,No Different Than the National Average Payment,20221.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Average Mortality and Average Payment,PN
18792,HOUSTON METHODIST THE WOODLANDS HOSPITAL,THE WOODLANDS,TX,77385,PAYM_30_AMI,Payment for heart attack patients,No Different Than the National Average Payment,27138.0,MORT_PAYM_30_AMI,Value of Care Heart Attack measure,Average Mortality and Average Payment,AMI
18793,HOUSTON METHODIST THE WOODLANDS HOSPITAL,THE WOODLANDS,TX,77385,PAYM_30_HF,Payment for heart failure patients,Greater Than the National Average Payment,21647.0,MORT_PAYM_30_HF,Value of Care Heart Failure measur,Average Mortality and Higher Payment,HF
18794,HOUSTON METHODIST THE WOODLANDS HOSPITAL,THE WOODLANDS,TX,77385,PAYM_30_PN,Payment for pneumonia patients,No Different Than the National Average Payment,19409.0,MORT_PAYM_30_PN,Value of Care Pneumonia measure,Better Mortality and Average Payment,PN


In [57]:
cleaned_df.to_csv("../Resources/Hospital.csv", encoding="utf-8", index=False)

In [58]:
new_df=pd.merge(cleaned_df, merged_df, on="ZIP Code")
new_df.columns
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11496 entries, 0 to 11495
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Facility Name                  11496 non-null  object 
 1   City                           11496 non-null  object 
 2   State                          11496 non-null  object 
 3   ZIP Code                       11496 non-null  int64  
 4   Payment Measure ID             11496 non-null  object 
 5   Payment Measure Name           11496 non-null  object 
 6   Payment Category               11496 non-null  object 
 7   Payment                        11496 non-null  float32
 8   Value of Care Display ID       11496 non-null  object 
 9   Value of Care Display Name     11496 non-null  object 
 10  Value of Care Category         11496 non-null  object 
 11  value_code                     11496 non-null  object 
 12  Population                     11496 non-null 

In [60]:
new_df['Value of Care Display ID'].unique()

array(['MORT_PAYM_30_AMI', 'MORT_PAYM_30_HF', 'MORT_PAYM_30_PN',
       'COMP_PAYM_90_HIP_KNEE'], dtype=object)

In [59]:
new_df.to_csv("../Resources/final.csv", encoding="utf-8", index=False)